In [ ]:
!pip install rouge
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.9 MB/s eta 0:00:00


In [ ]:
import json, gzip
from transformers import BartForConditionalGeneration, BartTokenizer
from rouge import Rouge
import random

In [ ]:
# model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
# tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-cnn-12-6")
tokenizer = BartTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
rouge = Rouge()

In [ ]:
path = "/content/test.jsonl.gz"
data = []

with gzip.open(path) as f:
    for ln in f:
        obj = json.loads(ln)
        data.append(obj)


In [ ]:
random.shuffle(data)
numArticles = 10
totScore_l = 0
totScore_1 = 0
totScore_2 = 0
for article in data[:numArticles]:
    text = article["text"]
    summary = article["summary"]
    # title = article["title"]

    inputs = tokenizer.batch_encode_plus([text], max_length=1024, return_tensors="pt")
    summary_ids = model.generate(
        inputs["input_ids"], num_beams=4, max_length=200, early_stopping=True
    )
    bart_summary = tokenizer.decode(summary_ids.squeeze(), skip_special_tokens=True)

    scores = rouge.get_scores(bart_summary, summary)
    totScore_l += scores[0]["rouge-l"]["f"]
    totScore_1 += scores[0]["rouge-1"]["f"]
    totScore_2 += scores[0]["rouge-2"]["f"]

print("Rouge_l")
print(totScore_l)
print(totScore_l / numArticles)
print()
print("Rouge_1")
print(totScore_1)
print(totScore_1 / numArticles)
print()
print("Rouge_2")
print(totScore_2)
print(totScore_2 / numArticles)

# print(title + ":")
# print(summary)
# print("BART Summary:")
# print(bart_summary)
# print()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Rouge_l
2.58228451047173
0.25822845104717296

Rouge_1
2.8099887516282704
0.28099887516282707

Rouge_2
1.3863618011934644
0.13863618011934645


In [ ]:
scores = rouge.get_scores("this is a summary", "this is a summary")
scores[0]

{'rouge-1': {'r': 1.0, 'p': 1.0, 'f': 0.999999995},
 'rouge-2': {'r': 1.0, 'p': 1.0, 'f': 0.999999995},
 'rouge-l': {'r': 1.0, 'p': 1.0, 'f': 0.999999995}}